# Modeling Pipeline: DWF Optimization with RL & Demand Forecasting
This notebook implements a Deep Reinforcement Learning model to dynamically optimize fare incentives using preprocessed and enriched ride-hailing data.

Import Libraries

In [66]:
# Load Libraries
import gymnasium as gym
from gymnasium import Env, spaces
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import VecNormalize
from scipy.special import expit
import os

Load Data

In [67]:
train_df = pd.read_csv("datasets/train_split.csv")
val_df =  pd.read_csv("datasets/val_split.csv")

In [68]:
# train_df.head(5)

Define RL Environment

In [ ]:
class RideHailingEnv_DWF(Env):
    def __init__(self, df):
        super(RideHailingEnv_DWF, self).__init__()
        self.df = df.reset_index(drop=True)
        self.current_idx = 0
        self.episode_limit = 1000
        self.episode_start = 0

        self.observation_space = spaces.Box(low=0, high=1, shape=(8,), dtype=np.float32)
        self.action_space = spaces.Box(
            low=np.array([-0.15, 0.0]),
            high=np.array([0.15, 5.0]),
            dtype=np.float32
        )

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.current_idx = np.random.randint(0, len(self.df) - self.episode_limit)
        self.episode_start = self.current_idx
        return self._get_observation(), {}

    def step(self, action):
        if self.current_idx >= len(self.df) - 1 or (self.current_idx - self.episode_start >= self.episode_limit):
            obs = self._get_observation()
            done = True
            reward = 0.0
            self.episode_start = self.current_idx + 1
            return obs, reward, done, False, {"CR": 0.5}

        row = self.df.iloc[self.current_idx]
        fare_adjustment, rider_incentive = np.round(action, 2)
        t_i = row['Request to Pickup']
        base_fare = row.get('Base Fare', 10.0)
        delta = 0.2 * base_fare

        # --- Behavior-based features
        rank_percentile = (self.df['Request to Pickup'] < t_i).sum() / len(self.df)
        epsilon = np.random.normal(loc=0.0, scale=0.02)
        RPI = np.clip(1.0 - rank_percentile + epsilon, 0.0, 1.0)

        # Amplify the impact of actions using exponential DPI
        action_signal = rider_incentive + abs(fare_adjustment * base_fare)
        DPI = np.clip(1.0 - np.exp(-0.6 * action_signal) + epsilon, 0.0, 1.0)

        # CR: stronger dependency on DPI (action-driven cancellation)
        cr_input = 1.2 * rank_percentile - 1.3 * RPI - 2.0 * DPI
        CR = 1.0 / (1.0 + np.exp(-cr_input))
        ride_completed = CR < 0.5

        # === Reward Components
        base_reward = 1.5 * (1.0 - CR)             # Completion-focused reward

        cost = rider_incentive + abs(fare_adjustment) * delta
        cost_ratio = cost / (base_fare + 5)
        cost_penalty = 0.5 * max(cost_ratio - 0.2, 0.0)  # Penalty only after 20%

        efficiency_bonus = 0.2 if ride_completed and cost_ratio < 0.2 else 0.0

        reward = base_reward - cost_penalty + efficiency_bonus
        reward = np.clip(reward, -2.0, 2.0)

        self.current_idx += 1
        done = False
        obs = self._get_observation()

        return obs, reward, done, False, {
            "CR": CR,
            "RPI": RPI,
            "DPI": DPI,
            "cost": cost,
            "base_reward": base_reward,
            "cost_penalty": cost_penalty
        }



    def _get_observation(self):
        if self.current_idx >= len(self.df):
            return np.zeros(self.observation_space.shape, dtype=np.float32)
        row = self.df.iloc[self.current_idx]
        return np.array([
            row['Pickup Location'],
            row['Request to Pickup'],
            row['Time of Day'],
            row['Month of Year'],
            row['RPI'],
            row['DPI'],
            row['CR'],
            row['Historical Demand Forecast']
        ], dtype=np.float32)


For Model Performance Monitoring

In [70]:
# Ensure logs directory exists
os.makedirs("logs", exist_ok=True)


# Training env
train_env = DummyVecEnv([lambda: RideHailingEnv_DWF(train_df)])
train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_reward=10.0)

# Validation env
val_env = DummyVecEnv([lambda: RideHailingEnv_DWF(val_df)])
val_env = VecNormalize(val_env, training=False, norm_obs=True, norm_reward=False)

# Evaluation callback
eval_callback = EvalCallback(
    val_env,
    best_model_save_path="./logs/",
    log_path="./logs/",
    eval_freq=5000,
    deterministic=True,
    render=False
)

Model Training

In [ ]:
from stable_baselines3 import PPO

# 9
model = PPO(
    "MlpPolicy",
    train_env,
    tensorboard_log="./ppo_tensorboard_logs/DWF/",
    learning_rate=1e-4,
    n_steps=2048,
    batch_size=256,
    gamma=0.98,
    gae_lambda=0.95,
    clip_range=0.4,
    ent_coef=0.01,
    vf_coef=0.8,               # boost critic learning slightly
    max_grad_norm=0.5,
    n_epochs=15,
    policy_kwargs=dict(net_arch=[64, 64]),
    verbose=1,
    device="cuda"
)

model.learn(total_timesteps=78000, callback=eval_callback)
model.save("models/dwf_model")

# Save normalization stats for later evaluation use
train_env.save("models/dwf_vecnormalize.pkl")


Using cuda device
Logging to ./ppo_tensorboard_logs/DWF_v11/PPO_1


c:\anaconda\envs\analysis\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 351  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 329         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005405333 |
|    clip_fraction        | 0.000358    |
|    clip_range           | 0.4         |
|    entropy_loss         | -2.84       |
|    explained_variance   | -0.57       |
|    learning_rate        | 0.0001      |
|    loss                 | 0.199       |
|    n_updates            | 15          |
|    policy_gradient_loss | -0.00494    |
|    std                  | 1           |
|    value_loss           | 0.628       |
----------------------------------

c:\anaconda\envs\analysis\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=5000, episode_reward=1880.22 +/- 18.25
Episode length: 1001.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 1e+03       |
|    mean_reward          | 1.88e+03    |
| time/                   |             |
|    total_timesteps      | 5000        |
| train/                  |             |
|    approx_kl            | 0.007157405 |
|    clip_fraction        | 0.00176     |
|    clip_range           | 0.4         |
|    entropy_loss         | -2.84       |
|    explained_variance   | -1.33       |
|    learning_rate        | 0.0001      |
|    loss                 | 0.107       |
|    n_updates            | 30          |
|    policy_gradient_loss | -0.00508    |
|    std                  | 1           |
|    value_loss           | 0.277       |
-----------------------------------------
New best mean reward!
-----------------------------
| time/              |      |
|    fps             | 217  |